## Table of Contents

- [0 - Preparation](#0)
    - [0.1 - Download LlaMA-Adpater V2 ](#0-1)
    - [0.2 - Packages](#0-2)
    - [0.3 - LlaMA Checkpoints](#0-3)
- [1 - Context Generation](#1)
    - [1.0 Load Llama-Adapter V2](#1-1)
    - [1.1 Load weight](#1-2)
- [2 - Text Clasification (RoBERTa)](#2)
- [3 - Evaluation](#3)  

In [1]:
INPUT_FOLDER = "test_acm"

In [2]:
# import zipfile

In [3]:
# zip_file = zipfile.ZipFile("test_acm.zip")
# zip_file.extractall()
# zip_file.close()

# 0 - Preparation

## 0.1 Download LLaMa-Adapter V2

In [4]:
!git clone https://github.com/OpenGVLab/LLaMA-Adapter.git

Cloning into 'LLaMA-Adapter'...
remote: Enumerating objects: 1151, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1151 (delta 9), reused 13 (delta 6), pack-reused 1126
Receiving objects: 100% (1151/1151), 34.41 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (541/541), done.


In [5]:
%cd LLaMA-Adapter/llama_adapter_v2_multimodal7b

/media/LLaMA-Adapter/llama_adapter_v2_multimodal7b


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
  Cloning https://github.com/csuhan/timm_0_3_2.git to /tmp/pip-req-build-wcsegbqh
  Running command git clone --filter=blob:none --quiet https://github.com/csuhan/timm_0_3_2.git /tmp/pip-req-build-wcsegbqh
  Resolved https://github.com/csuhan/timm_0_3_2.git to commit bf33f97f767a38e1b56cdd9c6a22372468fb55a4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-06h8e1pq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-06h8e1pq
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [14]:
%cd ..

/media/LLaMA-Adapter


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
%cd ..

/media


## 0.2 Install requirements package


In [7]:
!pip install gdown
!pip install torch
!pip install pandas
!pip install Pyarrow
!pip install transformers
!pip install opencv-python-headless
!pip install protobuf
!pip install datasets

### Import libraries


In [8]:
import cv2
from PIL import Image

import os
import matplotlib.pyplot as plt
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import gdown
import zipfile
import pandas as pd
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification


In [9]:
import os
import torch
import numpy as npval_test_df
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

### Load test file

In [10]:
def read_json(path: str) -> list:
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines()]

In [16]:
# Đường dẫn đến file test có context
json_test_file = INPUT_FOLDER + "/public_test_acm.json"
data = read_json(json_test_file)

# Chuyển đổi dữ liệu thành DataFrame
df_test = pd.DataFrame(data)

## 0.3 Llama-Adapter V2 checkpoints


In [17]:
!gdown --id 1LaQpjJsEdrD-9KhEEpYyXtx8HMsejjuR

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1LaQpjJsEdrD-9KhEEpYyXtx8HMsejjuR
From (redirected): https://drive.google.com/uc?id=1LaQpjJsEdrD-9KhEEpYyXtx8HMsejjuR&confirm=t&uuid=47dd432e-b0f5-431b-9e8b-095519027fd1
To: /media/weight-latest.zip
100%|██████████████████████████████████████| 12.4G/12.4G [03:53<00:00, 53.3MB/s]


In [18]:
zip_file = zipfile.ZipFile("weight-latest.zip")
zip_file.extractall()
zip_file.close()

# 1 - Context Generation

## 1.0 Load LLaMa-Adapter V2

In [19]:
%cd LLaMA-Adapter/llama_adapter_v2_multimodal7b

/media/LLaMA-Adapter/llama_adapter_v2_multimodal7b


In [20]:
import llama

In [21]:
%cd ..

/media/LLaMA-Adapter


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [22]:
%cd ..

/media


## 1.1 Load weight

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
llama_dir = "weight-latest/weight"

# choose from BIAS-7B, LORA-BIAS-7B, LORA-BIAS-7B-v21
model, preprocess = llama.load("LORA-BIAS-7B-v21", llama_dir, llama_type="7B", device=device)
model.eval()

100%|████████████████████████████████████████| 393M/393M [00:03<00:00, 111MiB/s]


Loading LLaMA-Adapter from ckpts/d26d107eec32127ac86ef1997cf7169de1c56a59c539fc1258c6798b969e289c_LORA-BIAS-7B-v21.pth


100%|███████████████████████████████████████| 890M/890M [00:20<00:00, 45.6MiB/s]


Trainable param: llama.layers.0.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.feed_forward.w2.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w3.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.attention_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.ffn_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.1.feed_forward.w2.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.feed_forward.

LLaMA_adapter(
  (clip): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQ

In [25]:

max_length = 1800  
def inference(image_path, prompt1, prompt2):
    # Handle image
    img1 = Image.fromarray(cv2.imread(image_path))
    img1 = preprocess(img1).unsqueeze(0).to(device)
    
    if len(prompt1) > max_length:
        prompt1 = prompt1[:max_length] 
    prompt1 = llama.format_prompt(prompt1)
    result1 = model.generate(img1, [prompt1])[0]

    img2 = Image.fromarray(cv2.imread(image_path))
    img2 = preprocess(img2).unsqueeze(0).to(device)
    if len(prompt2) > max_length:
        prompt2 = prompt2[:max_length] 
    prompt2 = llama.format_prompt(prompt2)
    result2 = model.generate(img2, [prompt2])[0]

    return result1, result2

In [ ]:
data = read_json(json_test_file)

# Define the full path to the images directory
images_directory = INPUT_FOLDER + "/images_test_acm"
for index, item in enumerate(data):
    print(index)
    image_filename = os.path.basename(item["img_local_path"])
    image_path = os.path.join(images_directory, image_filename)
    print(image_path)
    prompt1 = item["caption1"]
    prompt2 = item["caption2"]
    context1, context2 = inference(image_path, prompt1, prompt2)
    print(f"context 1: {context1}")
    print(f"context 2: {context2}")
    item["context1"] = context1
    item["context2"] = context2

0
test_acm/images_test_acm/0.jpg
context 1: In the image, a man is standing at a podium, giving a speech to a crowd of people. The man appears to be a politician, possibly Julian Castro, as he is wearing a suit and tie. The audience is engaged in the speech, with some of them holding flags, indicating that it might be a political event or rally. The man is speaking into a microphone, which is essential for amplifying his voice to reach the entire audience. The scene captures the public speaking aspect of politics and the importance of communication in conveying messages and ideas to the public.
context 2: In the image, a man is standing at a podium, giving a speech to a crowd of people. The man appears to be the main subject of the photo, and he is likely the speaker at an event. The crowd is gathered around the speaker, attentively listening to his words. There are several people in the audience, some standing closer to the speaker and others further back. The scene suggests that the 

In [ ]:
# Write the updated data back to the JSON file
with open('context_test.json', 'w') as file:
    json.dump(data, file)

# 2 - Text Classification

In [ ]:
import pandas as pd
import json

# Đường dẫn đến tệp JSON của bạn
json_file = "context_test.json"

# Đọc dữ liệu từ tệp JSON
with open(json_file, "r") as f:
    data = json.load(f)

# Chuyển đổi dữ liệu thành DataFrame
df = pd.DataFrame(data)

In [ ]:
df['context_combined'] = df['context1'] + "\n" + df['context2']

In [ ]:
test_sentences = df['context_combined'].tolist()

In [ ]:
model_id = "roberta-base"
repository_id = "hoanghoavienvo/roberta-base-detect-cheapfake-combined-train-test-15200-2-8"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(repository_id)
model_info = model.__dict__
print(model_info)

In [ ]:
model.training = False

# Chuẩn bị tokenizer
tokenizer = AutoTokenizer.from_pretrained(repository_id)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 4 - Evaluation

In [ ]:
predictions = []

# Chạy model trên tập test
for sentence in test_sentences:
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    inputs.to(device)
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=1)
    _, predicted_class = torch.max(probabilities, dim=1)
    predictions.append(predicted_class.item())

# Lưu kết quả vào DataFrame
results = pd.DataFrame({'text': test_sentences, 'predicted_label': predictions})

In [ ]:

from sklearn.metrics import classification_report

# Tính toán classification report
report = classification_report(df['context_label'], predictions, digits = 3)

# In ra classification report
print(report)